O objetivo desse notebook é a construção de um modelo capaz de predizer o valor do IBU (*International Bitterness Units*), que mensura o armagor, para um conjunto de cervejas.

Os dados utilizados no treinamento foram coletados da [PUNK API](https://punkapi.com/documentation/v2) e posteriormente tratados e armazenados em buckets S3 na AWS.

Para utilizá-los nesse notebook, será feito uma query SQL via Athena na tabela Glue criada na AWS, que retornará os dados coletados em um certo período de tempo da API.

# Bibliotecas

In [95]:
import os
import dotenv
import pickle
import time

import boto3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import requests
from xgboost import XGBRegressor

# Leitura dos dados

In [10]:
from pyathena import connect
dotenv.load_dotenv(".env")


S3_OUTPUT = "s3://punkapi-data-from-glue"
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
REGION_NAME = os.environ["REGION_NAME"]


conn = connect(aws_access_key_id=AWS_ACCESS_KEY_ID,
                 aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                 s3_staging_dir=S3_OUTPUT,
                 region_name=REGION_NAME)

query_string = (
    "SELECT abv, ebc, ibu, id, name, ph, srm, target_fg, target_og "
    "FROM punk_api_database.cleaned_data_from_punkapi"
)
df = pd.read_sql(query_string, conn)

In [16]:
df

,abv,ebc,ibu,id,name,ph,srm,target_fg,target_og
0,5.5,25.0,35,195,Hello My Name Is Holy Moose,4.4,12.50,1011,1053.0
1,7.5,9.0,35,280,Make Earth Great Again,4.6,5.00,1004,1065.0
2,8.0,240.0,45,101,Riptide,4.4,120.00,1014,1075.0
3,7.2,20.0,20,294,Opaque Jake,4.4,10.00,1009,1065.0
4,4.5,40.0,35,53,Baby Dogma,4.4,20.00,1013,1048.0
...,...,...,...,...,...,...,...,...,...
193,11.3,164.0,50,150,AB:13,4.4,83.00,1020,1098.0
194,7.1,15.0,90,124,Chaos Theory,4.4,7.50,1013,1067.0
195,7.0,170.0,25,259,Tropic Thunder,4.2,86.36,1020,1074.0
196,5.2,13.0,40,212,Hop Fiction - Prototype Challenge,4.4,6.50,1010,1048.0


# Pré-processamento do dado

Informações sobre o dado:

* ABV (Alcohol By Volume) - Quantidade de álcool presente na cerveja.
* EBC (European Brewery Convention) - Medida técnica europeia para a cor da cerveja.
* IBU (International Bitterness Units) - Índice que mede o amargor da cerveja.
* pH - Escala para determinar acidez/alcalinidade de uma solução.
* SRM (Standard Reference Method) - Sistema para medição de cor da cerveja usualmente aplicado nos Estados Unidos.
* Target FG (Final Gravity) - A gravidade final (FG) é a quantidade de açúcar que sobra quando a fermentação termina.
* Target OG (Original Gravity) - A gravidade original (OG) mede quanto açúcar está presente no mosto antes de ser fermentado. 

## Remoção de colunas

Duas colunas serão removidas do dado coletado: "id" e "name". A primeira, por se tratar apenas do identificador único de cada cerveja, não representa uma característica útil para determinar os IBU's. A segunda, apesar de poder ser transformada para um conjunto de valores numéricos via vetorização, será removida para simplificar o processo de treinamento.

In [18]:
INTEREST_COLUMNS = ["abv", "ebc", "ibu", "ph", "srm", "target_fg", "target_og"]
df_cleaned = df[INTEREST_COLUMNS]

In [19]:
df_cleaned

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,5.5,25.0,35,4.4,12.50,1011,1053.0
1,7.5,9.0,35,4.6,5.00,1004,1065.0
2,8.0,240.0,45,4.4,120.00,1014,1075.0
3,7.2,20.0,20,4.4,10.00,1009,1065.0
4,4.5,40.0,35,4.4,20.00,1013,1048.0
...,...,...,...,...,...,...,...
193,11.3,164.0,50,4.4,83.00,1020,1098.0
194,7.1,15.0,90,4.4,7.50,1013,1067.0
195,7.0,170.0,25,4.2,86.36,1020,1074.0
196,5.2,13.0,40,4.4,6.50,1010,1048.0


## Ajuste dos tipos dos valores

Para evitar exceções durante o treinamento do modelo, é importante que todas colunas contenham apenas valores numéricos. Como pode se ver abaixo, esse não é o caso para algumas colunas:

In [20]:
df_cleaned.dtypes

abv          float64
ebc          float64
ibu            int64
ph           float64
srm          float64
target_fg     object
target_og     object
dtype: object

In [21]:
df_cleaned = df_cleaned.apply(pd.to_numeric)

In [22]:
df_cleaned.dtypes

abv          float64
ebc          float64
ibu            int64
ph           float64
srm          float64
target_fg    float64
target_og    float64
dtype: object

## Remover linhas com NaN's

Também é necessário remover ou substituir linhas que possuem NaN's, pois são valores faltantes e podem impactar o treinamento dos modelos. Vamos checar quantos casos desses o dataset possui:

In [31]:
df_cleaned[df_cleaned.isna().any(axis=1)]

,abv,ebc,ibu,ph,srm,target_fg,target_og


Como é uma quantidade mínima de casos em relação ao conjunto total, vamos apenas remover essas linhas:

In [28]:
df_cleaned.dropna(inplace=True)

## Feature scaling

Para auxiliar o treinamento e melhorar a performance do modelo, é importante que a escala dos valores presentes no dado de treinamento esteja contida dentro de determinada faixa. Dessa forma, será usado uma normalização min-max que manterá os valores entre 0 e 1. Ela será incluída em uma pipeline do scikit-learn, juntamente com o modelo em si. Abaixo, uma amostra do que acontece ao aplicar a estratégia no dado coletado:

In [58]:
scaler = MinMaxScaler()
scaled_data = df_cleaned.copy()
scaled_data[INTEREST_COLUMNS] = scaler.fit_transform(df_cleaned)

In [151]:
scaled_data

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,0.138272,0.365000,0.041475,0.055696,0.359016,0.906667,0.927826
1,0.066667,0.011667,0.007373,0.040506,0.045902,0.895111,0.904348
2,0.158025,0.025000,0.032258,0.053165,0.024984,0.899556,0.926957
3,0.227160,0.050000,0.092166,0.055696,0.050164,0.902222,0.946087
4,0.362963,0.263333,0.018433,0.055696,0.259016,0.906667,0.965217
...,...,...,...,...,...,...,...
184,0.303704,0.185000,0.046083,0.055696,0.181967,0.906667,0.953043
185,0.385185,0.666667,0.078341,0.054430,0.655738,0.902222,0.978261
186,0.116049,0.013333,0.020276,0.051899,0.013115,0.896000,0.909565
187,0.165432,0.250000,0.046083,0.055696,0.249180,0.900444,0.928696


# Treinamento do modelo

Para realizar a tarefa de predizer dos valores de IBU, serão treinado um model de regressão linear.

In [25]:
X_COLUMNS = ["abv", "ebc", "ph", "srm", "target_fg", "target_og"]
Y_COLUMN = "ibu"

In [62]:
X, y = df_cleaned[X_COLUMNS], df_cleaned[Y_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [63]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', MinMaxScaler()), ('regressor', LinearRegression())])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [64]:
print(f"R2 (Reg. Linear): {r2_score(y_test, y_pred)}")

R2 (Reg. Linear): -2.2955162682703225


In [75]:
[[sample[var] for var in sample if var in X_COLUMNS] for sample in b]

[[5.3, 1012, 1052, 200, 100, 4.2], [9.2, 1016, 1085, 40, 20, 4.4]]

In [74]:
pipe.predict([[sample[var] for var in sample if var in X_COLUMNS] for sample in b])

array([25358.01687289, 26411.18194624])

# Armazenamento do modelo em um bucket S3

É possível acessar e realizar predições de maneira remota utilizando a infraestrutura da AWS. Para isso, o modelo treinado será armazendo em um bucket S3 da AWS e consumido via uma função lambda, acessada via API Gateway.


In [68]:
BUCKET_NAME = "beers-linear-regressor"
FILENAME = "pipeline.pkl"
FILE_PATH = os.path.join("/tmp/", FILENAME)

In [71]:
session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY_ID,
                        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                        region_name=REGION_NAME)

s3_client = session.client("s3")

with open(FILE_PATH, "wb") as f:
    pickle.dump(pipe, f)

s3_client.upload_file(FILE_PATH, BUCKET_NAME, FILENAME)

# Predições via lambda 

In [103]:
some_beers = requests.get(f"https://api.punkapi.com/v2/beers/?ids=42|77")

In [104]:
lambda_client = session.client('lambda')
response = lambda_client.invoke(
    FunctionName='predict_ibu',
    LogType='None',
    Payload=json.dumps({"data": some_beers.json()})
)

In [98]:
response

{'ResponseMetadata': {'RequestId': '61a6e06c-02fc-485e-9da1-65cd57d4e330',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 21 Oct 2021 06:54:02 GMT',
   'content-type': 'application/json',
   'content-length': '98',
   'connection': 'keep-alive',
   'x-amzn-requestid': '61a6e06c-02fc-485e-9da1-65cd57d4e330',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-61710e87-32b40aca5e6a7a2c6109ed85;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x7fa922907da0>}

In [105]:
res_json = json.loads(response['Payload'].read().decode("utf-8"))

In [100]:
res_json

{'statusCode': 200,
 'predictions': '[{"42": 26411.181946235152}, {"77": 25263.160643007883}]'}